# Regime Detection User Guide

This notebook provides a comprehensive guide to using the `algoshort.regimes` module for detecting market regimes and generating trading signals.

## Table of Contents

1. [Setup and Installation](#1-setup-and-installation)
2. [Understanding Regime Detection](#2-understanding-regime-detection)
3. [Unified RegimeDetector Interface](#3-unified-regimedetector-interface)
4. [Moving Average Crossover](#4-moving-average-crossover)
5. [Breakout Detection](#5-breakout-detection)
6. [Turtle Trader Strategy](#6-turtle-trader-strategy)
7. [Floor/Ceiling Swing Analysis](#7-floorceing-swing-analysis)
8. [Computing All Regimes](#8-computing-all-regimes)
9. [Complete Workflow Example](#9-complete-workflow-example)
10. [Best Practices and Tips](#10-best-practices-and-tips)

## 1. Setup and Installation

First, let's import the required modules.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import the unified RegimeDetector
from algoshort.regimes import RegimeDetector

# Or import individual detectors
from algoshort.regimes import (
    MovingAverageCrossover,
    BreakoutRegime,
    FloorCeilingRegime
)

# For data fetching
from algoshort.yfinance_handler import YFinanceDataHandler

print("Imports successful!")

## 2. Understanding Regime Detection

### What is a Regime?

A **market regime** is a classification of market conditions, typically:
- **Bullish (+1)**: Upward trend, favorable for long positions
- **Bearish (-1)**: Downward trend, favorable for short positions  
- **Neutral (0)**: Sideways/consolidation, unclear direction

### Available Methods

| Method | Description | Use Case |
|--------|-------------|----------|
| **SMA Crossover** | Triple Simple Moving Average crossover | Trend following, slower signals |
| **EMA Crossover** | Triple Exponential Moving Average crossover | Trend following, faster response |
| **Breakout** | Price breaking rolling high/low levels | Momentum, breakout trading |
| **Turtle Trader** | Dual-window breakout (slow + fast) | Classic trend-following system |
| **Floor/Ceiling** | Swing-based floor/ceiling detection | Support/resistance, regime changes |

In [ ]:
# See all available methods
methods = RegimeDetector.available_methods()
print("Available Regime Detection Methods:")
print("=" * 50)
for method, description in methods.items():
    print(f"{method:20s} : {description}")

## 3. Unified RegimeDetector Interface

The `RegimeDetector` class provides a single entry point for all regime detection methods.

In [ ]:
# Fetch sample data
handler = YFinanceDataHandler()
df = handler.get_ohlc_data('SPY', period='2y', interval='1d')

print(f"Downloaded {len(df)} rows")
df.head()

In [ ]:
# Create a RegimeDetector instance
detector = RegimeDetector(df)

print(detector)
print(f"\nDataFrame shape: {len(detector.df)} rows")

In [ ]:
# Use the generic compute() method with method names
result_sma = detector.compute('sma_crossover', short=10, medium=20, long=50)
result_bo = detector.compute('breakout', window=20)

# Method aliases also work
result_tt = detector.compute('tt', slow=50, fast=20)  # 'tt' = turtle

print("SMA columns:", [c for c in result_sma.columns if 'sma' in c])
print("Breakout columns:", [c for c in result_bo.columns if 'bo' in c or 'hi_' in c or 'lo_' in c])

## 4. Moving Average Crossover

### Triple MA Crossover Signal

The signal is the **product of two pairwise crossovers**:
- Short vs Medium crossover: +1 if short >= medium, -1 otherwise
- Medium vs Long crossover: +1 if medium >= long, -1 otherwise
- **Final Signal** = (Short vs Medium) × (Medium vs Long)

This means:
- **+1 (Bullish)**: Both crossovers bullish (short >= medium >= long)
- **-1 (Bearish)**: Both crossovers bearish (short <= medium <= long)
- **0 (Mixed)**: Crossovers disagree

In [ ]:
# Compute SMA crossover
result_sma = detector.sma_crossover(short=10, medium=20, long=50)

print("SMA Crossover Columns Added:")
sma_cols = [c for c in result_sma.columns if 'sma' in c.lower()]
for col in sma_cols:
    print(f"  - {col}")

# Display signal distribution
signal_col = 'sma_102050'
print(f"\nSignal Distribution ({signal_col}):")
print(result_sma[signal_col].value_counts().sort_index())

In [ ]:
# Compute EMA crossover (faster response)
result_ema = detector.ema_crossover(short=12, medium=26, long=50)

print("EMA Crossover Columns Added:")
ema_cols = [c for c in result_ema.columns if 'ema' in c.lower()]
for col in ema_cols:
    print(f"  - {col}")

signal_col = 'ema_122650'
print(f"\nSignal Distribution ({signal_col}):")
print(result_ema[signal_col].value_counts().sort_index())

In [ ]:
# Visualize SMA crossover
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Plot 1: Price and MAs
ax1 = axes[0]
ax1.plot(result_sma.index, result_sma['close'], label='Close', linewidth=1)
ax1.plot(result_sma.index, result_sma['sma_short_10'], label='SMA 10', linewidth=1)
ax1.plot(result_sma.index, result_sma['sma_medium_20'], label='SMA 20', linewidth=1)
ax1.plot(result_sma.index, result_sma['sma_long_50'], label='SMA 50', linewidth=1)
ax1.set_ylabel('Price ($)')
ax1.set_title('SPY - Price with Triple SMA')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Signal
ax2 = axes[1]
signal = result_sma['sma_102050']
ax2.fill_between(result_sma.index, 0, signal, where=signal > 0, 
                 color='green', alpha=0.5, label='Bullish')
ax2.fill_between(result_sma.index, 0, signal, where=signal < 0, 
                 color='red', alpha=0.5, label='Bearish')
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Signal')
ax2.set_xlabel('Date')
ax2.set_title('Triple SMA Crossover Signal')
ax2.legend(loc='upper left')
ax2.set_ylim(-1.5, 1.5)

plt.tight_layout()
plt.show()

## 5. Breakout Detection

### Single-Window Breakout

- **+1 (Bullish)**: High equals rolling max high (new high breakout)
- **-1 (Bearish)**: Low equals rolling min low (new low breakdown)
- Signal is forward-filled to maintain position

In [ ]:
# Compute breakout with 20-day window
result_bo = detector.breakout(window=20)

print("Breakout Columns Added:")
bo_cols = [c for c in result_bo.columns if 'bo_' in c or 'hi_' in c or 'lo_' in c]
for col in bo_cols:
    print(f"  - {col}")

print(f"\nSignal Distribution (bo_20):")
print(result_bo['bo_20'].value_counts().sort_index())

In [ ]:
# Visualize breakout
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Plot 1: Price with rolling high/low
ax1 = axes[0]
ax1.plot(result_bo.index, result_bo['close'], label='Close', linewidth=1)
ax1.plot(result_bo.index, result_bo['hi_20'], 'g--', label='20-day High', linewidth=1, alpha=0.7)
ax1.plot(result_bo.index, result_bo['lo_20'], 'r--', label='20-day Low', linewidth=1, alpha=0.7)

# Mark breakout points
breakout_up = (result_bo['high'] == result_bo['hi_20']) & (result_bo['bo_20'] == 1)
breakout_dn = (result_bo['low'] == result_bo['lo_20']) & (result_bo['bo_20'] == -1)
ax1.scatter(result_bo.index[breakout_up], result_bo.loc[breakout_up, 'high'], 
            marker='^', color='green', s=50, label='Breakout Up', zorder=5)
ax1.scatter(result_bo.index[breakout_dn], result_bo.loc[breakout_dn, 'low'], 
            marker='v', color='red', s=50, label='Breakout Down', zorder=5)

ax1.set_ylabel('Price ($)')
ax1.set_title('SPY - 20-Day Breakout')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Signal
ax2 = axes[1]
signal = result_bo['bo_20']
ax2.fill_between(result_bo.index, 0, signal, where=signal > 0, 
                 color='green', alpha=0.5, label='Bullish')
ax2.fill_between(result_bo.index, 0, signal, where=signal < 0, 
                 color='red', alpha=0.5, label='Bearish')
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Signal')
ax2.set_xlabel('Date')
ax2.set_title('Breakout Signal')
ax2.legend()
ax2.set_ylim(-1.5, 1.5)

plt.tight_layout()
plt.show()

## 6. Turtle Trader Strategy

### Dual-Window Breakout

The Turtle Trader combines two breakout windows:
- **Slow window** (e.g., 50 days): Entry confirmation
- **Fast window** (e.g., 20 days): Timing signals

Signal logic:
- **+1 (Long)**: Both slow AND fast are bullish
- **-1 (Short)**: Both slow AND fast are bearish
- **0 (Neutral)**: Mixed signals

In [ ]:
# Compute Turtle Trader with 50/20 windows
result_tt = detector.turtle(slow=50, fast=20)

print("Turtle Trader Columns Added:")
tt_cols = [c for c in result_tt.columns if 'tt_' in c or 'bo_' in c or 'hi_' in c or 'lo_' in c]
for col in sorted(tt_cols):
    print(f"  - {col}")

print(f"\nSignal Distribution (tt_5020):")
print(result_tt['tt_5020'].value_counts().sort_index())

In [ ]:
# Visualize Turtle Trader
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Plot 1: Price with channels
ax1 = axes[0]
ax1.plot(result_tt.index, result_tt['close'], label='Close', linewidth=1)
ax1.plot(result_tt.index, result_tt['hi_50'], 'g-', label='50-day High', linewidth=1, alpha=0.5)
ax1.plot(result_tt.index, result_tt['lo_50'], 'r-', label='50-day Low', linewidth=1, alpha=0.5)
ax1.fill_between(result_tt.index, result_tt['hi_20'], result_tt['lo_20'], 
                 alpha=0.2, color='blue', label='20-day Channel')
ax1.set_ylabel('Price ($)')
ax1.set_title('SPY - Turtle Trader Channels (50/20)')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Individual breakout signals
ax2 = axes[1]
ax2.plot(result_tt.index, result_tt['bo_50'], label='Slow (50)', linewidth=1)
ax2.plot(result_tt.index, result_tt['bo_20'], label='Fast (20)', linewidth=1, alpha=0.7)
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Signal')
ax2.set_title('Individual Breakout Signals')
ax2.legend()
ax2.set_ylim(-1.5, 1.5)

# Plot 3: Combined Turtle signal
ax3 = axes[2]
signal = result_tt['tt_5020']
ax3.fill_between(result_tt.index, 0, signal, where=signal > 0, 
                 color='green', alpha=0.5, label='Long')
ax3.fill_between(result_tt.index, 0, signal, where=signal < 0, 
                 color='red', alpha=0.5, label='Short')
# Mark neutral zones
neutral = signal == 0
ax3.scatter(result_tt.index[neutral], [0] * neutral.sum(), 
            marker='|', color='gray', alpha=0.3, s=10)
ax3.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax3.set_ylabel('Signal')
ax3.set_xlabel('Date')
ax3.set_title('Turtle Trader Combined Signal')
ax3.legend()
ax3.set_ylim(-1.5, 1.5)

plt.tight_layout()
plt.show()

## 7. Floor/Ceiling Swing Analysis

### Swing-Based Regime Detection

This sophisticated method:
1. Identifies swing highs and lows at multiple levels
2. Detects floor (support) and ceiling (resistance) levels
3. Tracks regime changes when floors/ceilings are broken

**Note**: This method is more complex and may produce different results based on parameters.

In [ ]:
# Compute Floor/Ceiling regime
# Note: Using WARNING log level to reduce output
detector_fc = RegimeDetector(df, log_level=logging.WARNING)
result_fc = detector_fc.floor_ceiling(lvl=1, threshold=1.5)

print("Floor/Ceiling Columns Added:")
fc_cols = [c for c in result_fc.columns if any(x in c for x in ['rg', 'flr', 'clg', 'hi', 'lo'])]
for col in sorted(fc_cols)[:15]:  # Show first 15
    print(f"  - {col}")

print(f"\nRegime Signal Distribution (rg):")
print(result_fc['rg'].value_counts().sort_index())

In [ ]:
# Visualize Floor/Ceiling
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Plot 1: Price with floors and ceilings
ax1 = axes[0]
ax1.plot(result_fc.index, result_fc['close'], label='Close', linewidth=1)

# Plot floor levels
if 'flr' in result_fc.columns:
    floors = result_fc['flr'].dropna()
    ax1.scatter(floors.index, floors.values, marker='^', color='green', 
                s=100, label='Floor', zorder=5)
    # Extend floor lines
    for idx, val in floors.items():
        end_idx = min(result_fc.index[-1], idx + pd.Timedelta(days=30))
        ax1.hlines(y=val, xmin=idx, xmax=end_idx, colors='green', 
                   linestyles='--', alpha=0.5)

# Plot ceiling levels
if 'clg' in result_fc.columns:
    ceilings = result_fc['clg'].dropna()
    ax1.scatter(ceilings.index, ceilings.values, marker='v', color='red', 
                s=100, label='Ceiling', zorder=5)
    # Extend ceiling lines
    for idx, val in ceilings.items():
        end_idx = min(result_fc.index[-1], idx + pd.Timedelta(days=30))
        ax1.hlines(y=val, xmin=idx, xmax=end_idx, colors='red', 
                   linestyles='--', alpha=0.5)

ax1.set_ylabel('Price ($)')
ax1.set_title('SPY - Floor/Ceiling Levels')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Regime signal
ax2 = axes[1]
signal = result_fc['rg']
ax2.fill_between(result_fc.index, 0, signal, where=signal > 0, 
                 color='green', alpha=0.5, label='Bullish')
ax2.fill_between(result_fc.index, 0, signal, where=signal < 0, 
                 color='red', alpha=0.5, label='Bearish')
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Regime')
ax2.set_xlabel('Date')
ax2.set_title('Floor/Ceiling Regime Signal')
ax2.legend()
ax2.set_ylim(-1.5, 1.5)

plt.tight_layout()
plt.show()

## 8. Computing All Regimes

The `compute_all()` method computes all regime signals at once.

In [ ]:
# Compute all regimes with default parameters
detector_all = RegimeDetector(df, log_level=logging.WARNING)

result_all = detector_all.compute_all(
    sma_params={'short': 10, 'medium': 20, 'long': 50},
    ema_params={'short': 12, 'medium': 26, 'long': 50},
    breakout_params={'window': 20},
    turtle_params={'slow': 50, 'fast': 20},
    floor_ceiling_params={'lvl': 1, 'threshold': 1.5}
)

print(f"Total columns: {len(result_all.columns)}")
print(f"\nNew columns added: {len(result_all.columns) - len(df.columns)}")

In [ ]:
# Get signal column names
signal_cols = detector_all.get_signal_columns(
    sma_params={'short': 10, 'medium': 20, 'long': 50},
    ema_params={'short': 12, 'medium': 26, 'long': 50},
    breakout_window=20,
    turtle_params={'slow': 50, 'fast': 20}
)

print("Signal Columns:")
for method, col in signal_cols.items():
    print(f"  {method:20s}: {col}")

In [ ]:
# Compare all signals
signal_comparison = pd.DataFrame()
for method, col in signal_cols.items():
    if col in result_all.columns:
        signal_comparison[method] = result_all[col]

# Calculate agreement
signal_comparison['agreement'] = signal_comparison.apply(
    lambda row: 1 if len(set(row.dropna())) <= 1 else 0, axis=1
)

print("Signal Agreement Analysis:")
print(f"All methods agree: {signal_comparison['agreement'].mean()*100:.1f}% of the time")
print(f"\nSignal correlation:")
print(signal_comparison.drop('agreement', axis=1).corr().round(2))

In [ ]:
# Plot all signals comparison
fig, axes = plt.subplots(len(signal_cols) + 1, 1, figsize=(14, 12), sharex=True)

# Price
axes[0].plot(result_all.index, result_all['close'], 'b-', linewidth=1)
axes[0].set_ylabel('Price')
axes[0].set_title('SPY - All Regime Signals Comparison')
axes[0].grid(True, alpha=0.3)

# Each signal
for i, (method, col) in enumerate(signal_cols.items(), 1):
    if col in result_all.columns:
        signal = result_all[col]
        axes[i].fill_between(result_all.index, 0, signal, where=signal > 0, 
                             color='green', alpha=0.5)
        axes[i].fill_between(result_all.index, 0, signal, where=signal < 0, 
                             color='red', alpha=0.5)
        axes[i].axhline(y=0, color='gray', linestyle='--', alpha=0.3)
        axes[i].set_ylabel(method[:10])
        axes[i].set_ylim(-1.5, 1.5)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

## 9. Complete Workflow Example

Let's put it all together with a complete trading workflow.

In [ ]:
# Complete workflow
from algoshort import (
    YFinanceDataHandler,
    OHLCProcessor,
    StopLossCalculator,
    ReturnsCalculator,
    PositionSizing,
    RegimeDetector
)

# Step 1: Get data
handler = YFinanceDataHandler()
symbols = ['AAPL', 'MSFT', 'GOOGL']
data = {}

for symbol in symbols:
    data[symbol] = handler.get_ohlc_data(symbol, period='2y', interval='1d')
    print(f"Downloaded {symbol}: {len(data[symbol])} rows")

In [ ]:
# Step 2: Compute regime signals for each symbol
regime_signals = {}

for symbol, df in data.items():
    detector = RegimeDetector(df, log_level=logging.WARNING)
    
    # Use SMA crossover as the signal
    result = detector.sma_crossover(short=10, medium=20, long=50)
    
    # Store signal
    regime_signals[symbol] = result['sma_102050']
    
    # Signal statistics
    bullish_pct = (result['sma_102050'] == 1).mean() * 100
    bearish_pct = (result['sma_102050'] == -1).mean() * 100
    print(f"{symbol}: {bullish_pct:.1f}% bullish, {bearish_pct:.1f}% bearish")

In [ ]:
# Step 3: Calculate stop-loss for one symbol
symbol = 'AAPL'
df_workflow = data[symbol].copy()

# Add signal column
detector = RegimeDetector(df_workflow, log_level=logging.WARNING)
df_workflow = detector.sma_crossover(short=10, medium=20, long=50)

# Calculate ATR stop-loss
stop_calc = StopLossCalculator(df_workflow)
df_workflow = stop_calc.atr_stop_loss(
    signal='sma_102050',
    multiplier=2.0,
    window=14,
    forward_fill=True
)

print(f"Stop-loss column added: sma_102050_atr_stop_loss")
df_workflow[['close', 'sma_102050', 'sma_102050_atr_stop_loss']].tail()

In [ ]:
# Step 4: Calculate returns
returns_calc = ReturnsCalculator()
df_workflow = returns_calc.calculate_returns(
    df=df_workflow,
    signals=['sma_102050'],
    inplace=False
)

# Prepare for position sizing
df_workflow['sma_102050_chg1D_fx'] = df_workflow['close'].diff().fillna(0)
df_workflow['sma_102050_stop_loss'] = df_workflow['sma_102050_atr_stop_loss']

print("Returns calculated")
df_workflow = df_workflow.dropna().reset_index(drop=True)

In [ ]:
# Step 5: Position sizing
sizer = PositionSizing(
    tolerance=-0.15,
    mn=0.01,
    mx=0.03,
    equal_weight=0.10,
    avg=0.02,
    lot=1,
    initial_capital=100000
)

df_final = sizer.calculate_shares(
    df=df_workflow,
    signal='sma_102050',
    daily_chg='sma_102050_chg1D_fx',
    sl='sma_102050_stop_loss',
    close='close'
)

print("Position sizing complete!")

In [ ]:
# Final visualization
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Plot 1: Price and signal
ax1 = axes[0]
ax1.plot(df_final.index, df_final['close'], 'b-', linewidth=1, label='Close')
# Color background by regime
signal = df_final['sma_102050']
ax1.fill_between(df_final.index, df_final['close'].min(), df_final['close'].max(),
                 where=signal > 0, alpha=0.2, color='green', label='Bullish')
ax1.fill_between(df_final.index, df_final['close'].min(), df_final['close'].max(),
                 where=signal < 0, alpha=0.2, color='red', label='Bearish')
ax1.set_ylabel('Price ($)')
ax1.set_title(f'{symbol} - Complete Workflow Results')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Equity curves
ax2 = axes[1]
equity_cols = [c for c in df_final.columns if 'sma_102050_equity' in c]
for col in equity_cols:
    strategy = col.split('_')[-1].title()
    ax2.plot(df_final.index, df_final[col], label=strategy, linewidth=1.5)
ax2.axhline(y=100000, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Equity ($)')
ax2.set_title('Portfolio Equity by Strategy')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

# Plot 3: Drawdown
ax3 = axes[2]
for col in equity_cols:
    equity = df_final[col]
    drawdown = (equity / equity.expanding().max() - 1) * 100
    strategy = col.split('_')[-1].title()
    ax3.fill_between(df_final.index, 0, drawdown, alpha=0.3, label=strategy)
ax3.set_ylabel('Drawdown (%)')
ax3.set_xlabel('Period')
ax3.set_title('Strategy Drawdowns')
ax3.legend(loc='lower left')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final statistics
print(f"\n{'='*60}")
print(f"PERFORMANCE SUMMARY - {symbol}")
print(f"{'='*60}")
print(f"\n{'Strategy':<15} {'Final Equity':>15} {'Return':>10} {'Max DD':>10}")
print("-" * 55)
for col in equity_cols:
    strategy = col.split('_')[-1].title()
    final_eq = df_final[col].iloc[-1]
    total_ret = (final_eq - 100000) / 100000 * 100
    max_dd = (df_final[col] / df_final[col].expanding().max() - 1).min() * 100
    print(f"{strategy:<15} ${final_eq:>13,.2f} {total_ret:>+9.2f}% {max_dd:>9.2f}%")

## 10. Best Practices and Tips

### Parameter Selection Guidelines

| Method | Conservative | Moderate | Aggressive |
|--------|-------------|----------|------------|
| **SMA** | 20/50/200 | 10/20/50 | 5/10/20 |
| **EMA** | 12/26/50 | 8/21/50 | 5/13/26 |
| **Breakout** | 50 days | 20 days | 10 days |
| **Turtle** | 55/20 | 50/20 | 40/10 |

### Common Pitfalls

1. **Overfitting**: Don't optimize parameters too aggressively
2. **Look-ahead bias**: Signals should only use past data
3. **Survivorship bias**: Test on complete universes
4. **Regime changes**: Methods perform differently in different market conditions

### Performance Considerations

In [ ]:
# Caching demonstration - second call is faster
import time

detector = RegimeDetector(df)

# First call (computes MAs)
start = time.time()
_ = detector.sma_crossover(short=10, medium=20, long=50)
first_time = time.time() - start

# Second call (uses cache)
start = time.time()
_ = detector.sma_crossover(short=10, medium=20, long=50)
second_time = time.time() - start

print(f"First call:  {first_time*1000:.2f} ms")
print(f"Second call: {second_time*1000:.2f} ms")
print(f"Speedup: {first_time/second_time:.1f}x (due to caching)")

### Quick Reference

```python
# Import
from algoshort.regimes import RegimeDetector

# Create detector
detector = RegimeDetector(df)

# MA Crossover
result = detector.sma_crossover(short=5, medium=10, long=20)
result = detector.ema_crossover(short=12, medium=26, long=50)

# Breakout
result = detector.breakout(window=20)
result = detector.turtle(slow=50, fast=20)

# Floor/Ceiling
result = detector.floor_ceiling(lvl=1, threshold=1.5)

# All at once
result = detector.compute_all()

# Generic compute
result = detector.compute('sma_crossover', short=10, medium=20, long=50)
```

---

## Summary

This guide covered:

1. **Unified Interface**: `RegimeDetector` provides single entry point
2. **MA Crossover**: Triple SMA/EMA crossover for trend following
3. **Breakout**: Single-window momentum detection
4. **Turtle Trader**: Classic dual-window trend-following system
5. **Floor/Ceiling**: Sophisticated swing-based regime detection
6. **Batch Processing**: `compute_all()` for multiple signals
7. **Integration**: Complete workflow with other algoshort modules

For more details, refer to:
- Test suite: `tests/test_regimes.py`
- Source code: `algoshort/regimes/`